# Gun Charge Takeaways Post Bruen Ruling

Description

In [7]:
#imports
import datetime
from itertools import product
import os
import pandas as pd
import urllib.request
from zipfile import ZipFile

### Import pretrial data

Retrieve NYC and ONYC pretrial datasets from the NYS Unified Court System webpage. 

Datasets: https://ww2.nycourts.gov/pretrial-release-data-33136
<br>urllib doc: https://docs.python.org/3.12/library/urllib.request.html

In [8]:
#Use agent given in urllib doc to bypass default filters
agent = "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11" 
reader = urllib.request.build_opener()
reader.addheaders = [('User-Agent', agent)]
urllib.request.install_opener(reader)

onyc_path = r"https://www.nycourts.gov/legacypdfs/court-research/ONYC%20for%20Web.zip"
nyc_path = r"https://www.nycourts.gov/legacypdfs/court-research/NYC%20for%20Web.zip"

#Make pretrial directory if needed
if not os.path.exists("pretrial"):
    os.mkdir("pretrial")
    
#Download NYC pretrial data
urllib.request.urlretrieve(nyc_path, "pretrial/nyc.zip")
with ZipFile("pretrial/nyc.zip", "r") as zip_file:
    zip_file.getinfo(r"NYC for Web.csv").filename = "nyc.csv"
    zip_file.extract(r"NYC for Web.csv", path=r"pretrial")
    zip_file.close()
os.remove("pretrial/nyc.zip")

#Download ONYC pretrial data
urllib.request.urlretrieve(onyc_path, "pretrial/onyc.zip")
with ZipFile("pretrial/onyc.zip", "r") as zip_file:
    zip_file.getinfo(r"ONYC for Web.csv").filename = "onyc.csv"
    zip_file.extract(r"ONYC for Web.csv", path=r"pretrial")
    zip_file.close()
os.remove("pretrial/onyc.zip")

### Create unified dataframe 

Concatenate the NYC and ONYC data into a single dataframe with necessary column values. 

In [9]:
columns = ["Internal_Case_ID", "Race", "Ethnicity", "County_Name",
           "Region", "First_Arraign_Date", "Top_Severity_at_Arraign",
           "Top_Arraign_Article_Section", "Representation_Type", 
           "Remanded_to_Jail_at_Arraign", "ROR_at_Arraign",
           "Bail_Set_and_Posted_at_Arraign", 
           "Bail_Set_and_Not_Posted_at_Arraign",
           "NMR_at_Arraign", r"Release Decision at Arraign", 
           "prior_vfo_cnt", "prior_nonvfo_cnt", "prior_misd_cnt"]
nyc_data = pd.read_csv("pretrial/nyc.csv", usecols=columns)
onyc_data = pd.read_csv("pretrial/onyc.csv", usecols=columns)
pretrial = pd.concat([nyc_data, onyc_data], ignore_index=True)

## CWP and felony charges by region and demographic

Create a breakdown of pretrial CWP cases and overal felony cases by region (Bronx, NYC, NYS) and race (Black, White/Non-Hispanic, all). Include cases arraigned since the Bruen ruling (beginning July 2022) and with defendants with no criminal history.

<u>Computed values:</u>
<br>&emsp;&emsp;&emsp;- Total Cases
<br>&emsp;&emsp;&emsp;- % of regional total
<br>&emsp;&emsp;&emsp;- % bail set or remanded
<br>&emsp;&emsp;&emsp;- % ror
<br>&emsp;&emsp;&emsp;- % made bail
<br>&emsp;&emsp;&emsp;- % assigned counsel

#### Dictionaries and helper functions

Create maps from race/region in results data to race/ethnicity/region values in pretrial dataframe. Create helper function to compute values for each race/region row.

In [10]:
'''
Conditions to map race/region values in results to corresponding race/ethnicity/region 
values in pretrial data
'''
region_map = {
    "Bronx" : (lambda n : n == "Bronx"),
    "NYC" : (lambda n : n == ("NYC" or "Bronx")), 
    "NYS" : (lambda n : True)
}

race_map = {
    "All" : (lambda n : True, lambda n : True),
    "Black" : (lambda n : n == "Black", lambda n : True),
    "White/Non-Hispanic" : (lambda n : n == "White", 
                            lambda n : n == "Non Hispanic")
}

#Keep track of totals per region for % total calculation
region_totals = {
    "Bronx" : 0,
    "NYC" : 0,
    "NYS" : 0
}

region_totals_cwp = {
    "Bronx" : 0,
    "NYC" : 0,
    "NYS" : 0
}
    

def filter_data(data, date_start, date_end=datetime.datetime.today()):
    """Filter data for cases arraigned since the Bruen ruling (beginning 
    July 2022) and for defendants with no criminal history.

    FIX: TOO MUCH MEMORY (PROBABLY)

    Args:
        data (DataFrame): pretrial data to filter
        date_start: beginning of timeframe
        date_end: end of timeframe

    Returns:
        DataFrame: the filtered data
    """
    #Filter for cases arraigned since July 2022
    data["First_Arraign_Date"] = pd.to_datetime(data["First_Arraign_Date"])
    data = data[(data["First_Arraign_Date"] >= date_start)
                & (data["First_Arraign_Date"] < date_end)]
            
    # #Filter for defendants with no criminal history
    # data = data[(data["prior_vfo_cnt"].apply(lambda n : pd.isna(n) or n == 0))
    #             & (data["prior_nonvfo_cnt"].apply(lambda n : pd.isna(n) or n == 0))
    #             & (data["prior_misd_cnt"].apply(lambda n : pd.isna(n) or n == 0))]

    #Set region to Bronx for Bronx cases
    data.loc[data["County_Name"] == "Bronx", "Region"] = "Bronx"

    data.reset_index(drop=True, inplace=True)

    return data


def get_row_data(row, filtered_pretrial):
    """Generates pretrial figures by region. Determines total count, percentage of
    regional total by race, % bail or remand, % 

    Args:
        row (list): A row in the results table with format [region, race]
        filtered_pretrial (dataframe): the frame to search

    Returns:
        list: A list of values to append to the row. Includes total count, % of
        regional total, % bail/remand, % ror, % made bail, and % assigned counsel.
    """
    region = row[0]
    race_eth = row[1]

    '''
    Select subset of pretrial data corresponding to specified region & race.
    Use mapping dictionaries to retrieve equivalent race/region values in 
    pretrial dataframe.
    '''
    pretrial_data = filtered_pretrial[(filtered_pretrial["Region"].apply(region_map[region]))
                    & (filtered_pretrial["Race"].apply(race_map[race_eth][0]))
                    & (filtered_pretrial["Ethnicity"].apply(race_map[race_eth][1]))]
    # Get total count
    total_count = pretrial_data.index.size
    #Add to region total map if row includes all races/ethnicities
    if (race_eth == "All"):
        region_totals[region] = total_count

    # Get % of total
    percent_of_total = f"{total_count / region_totals[region]:.2%}"

    # Get % bail/remand
    bail_or_remand = ((pretrial_data["Remanded_to_Jail_at_Arraign"] == "Y")
                      | (pretrial_data["Bail_Set_and_Posted_at_Arraign"] == "Y")
                      | (pretrial_data["Bail_Set_and_Not_Posted_at_Arraign"] == "Y")
                      | (pretrial_data["Release Decision at Arraign"] == "Bail-set"))
    bail_remand_count = pretrial_data[bail_or_remand].index.size
    bail_remand_percent = f"{bail_remand_count / total_count:.2%}"

    # Get % ror
    ror_count = pretrial_data[pretrial_data["ROR_at_Arraign"] == "Y"].index.size
    ror_percent = f"{ror_count / total_count:.2%}"

    # Get % nmr
    nmr_count = pretrial_data[pretrial_data["NMR_at_Arraign"] == "Y"].index.size
    nmr_percent = f"{nmr_count / total_count:.2%}"

    # Get % made bail
    made_bail = pretrial_data["Bail_Set_and_Posted_at_Arraign"] == "Y"
    made_bail_count = pretrial_data[made_bail].index.size
    made_bail_percent = f"{made_bail_count / total_count:.2%}"

    # Get % assigned counsel
    assigned_counsel = pretrial_data["Representation_Type"].apply(lambda n :
                       n in ['Legal Aid', '18B (Assigned Counsel)', 'Public Defender',
                             'Conflict Defender'])
    assigned_counsel_count = pretrial_data[assigned_counsel].index.size
    assigned_counsel_percent = f"{assigned_counsel_count / total_count:.2%}"
    
    # Return computed values
    return [total_count, percent_of_total, bail_remand_percent, ror_percent, 
            nmr_percent, made_bail_percent, assigned_counsel_percent]

#### Filter 

Create two dataframes for CWP cases and felony cases. Use filter function to filter for cases arraigned since July 2022 and defendants with no criminal history. 

We looked at the data itself to find and include all appearing representations of the desired CWP charges.

<u>CWP charges to include:</u>
<br>&emsp;&emsp;&emsp;- 265.01-B: 4th degree criminal possession of a firearm
<br>&emsp;&emsp;&emsp;- 265.02: 3rd degree criminal possession of a firearm
<br>&emsp;&emsp;&emsp;- 265.03: 2nd degree criminal possession of a firearm
<br>&emsp;&emsp;&emsp;- 265.04: 1st degree criminal possession of a firearm

#### Generate CWP and felony pretrial data by region

Use the helper function to generate results data.

In [15]:
dates = ["01/01/2020", "02/01/2020", "03/01/2020", "04/01/2020", "05/01/2020", 
         "06/01/2020", "07/01/2020", "08/01/2020", "09/01/2020", "10/01/2020", 
         "11/01/2020", "12/01/2020", "1/01/2021", "02/01/2021", "03/01/2021", 
         "04/01/2021", "05/01/2021", "06/01/2021", "07/01/2021", "08/01/2021", 
         "09/01/2021", "10/01/2021", "11/01/2021", "12/01/2021", "01/01/2022", 
         "02/01/2022", "03/01/2022", "04/01/2022", "05/01/2022", "06/01/2022", 
         "07/01/2022", "08/01/2022", "09/01/2022", "10/01/2022", "11/01/2022", 
         "12/01/2022", "01/01/2023", "02/01/2023", "03/01/2023", "04/01/2023", 
         "05/01/2023", "06/01/2023"]
num_month = 0

for i in len(dates):
    if num_month < i:
        start = dates[num_month]
        end = dates[num_month+1]
        num_month =+ 1


#Filter for desired CWP charges
# charges = ["265.01","265.02","265.03","265.04","265.1","265.01-B"]
charges = ["265.02","265.03","265.04","265.01-B"]
cwp = pretrial[(pretrial["Top_Arraign_Article_Section"] == "265.01-B")
               | (pretrial["Top_Arraign_Article_Section"] == "265.02")
               | (pretrial["Top_Arraign_Article_Section"] == "265.03")
               | (pretrial["Top_Arraign_Article_Section"] == "265.04")]
cwp = filter_data(cwp, start, end)

#Filter for felonies
felonies = pretrial[pretrial["Top_Severity_at_Arraign"] == "Felony"]
felonies = filter_data(felonies, start, end)

#Create data lists
entries = list(product(["Bronx", "NYC", "NYS"], ["All", "Black", "White/Non-Hispanic"]))

cwp_data = [[entry[0], entry[1]] for entry in entries] 
felony_data = [[entry[0], entry[1]] for entry in entries]

#Generate rows. Note: cannot do this simultaneously, or total % will be inaccurate
for n, row in enumerate(cwp_data):
    row.extend(get_row_data(row, cwp))

for n, row in enumerate(felony_data):
    row.extend(get_row_data(row, felonies))

#Create results dfs
columns = ["Region", "Race/Ethnicity", "Total Count", "Percent of Total",
           "% Bail Set or Remanded", "% ROR", "% NMR", "% Made Bail",
           "% Assigned Counsel"]

cwp_results = pd.DataFrame(data = cwp_data, columns = columns)
felony_results = pd.DataFrame(data = felony_data, columns = columns)

/var/folders/08/db88zh6x7419ybprdyc3dz1h0000gn/T/ipykernel_28732/1454356600.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["First_Arraign_Date"] = pd.to_datetime(data["First_Arraign_Date"])
/var/folders/08/db88zh6x7419ybprdyc3dz1h0000gn/T/ipykernel_28732/1454356600.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["First_Arraign_Date"] = pd.to_datetime(data["First_Arraign_Date"])


#### Export results

Export results as CSV files to results directory. CWP results to cwp_by_region.csv, felony results to felonies_by_region.csv.

In [12]:
#Create results directory
if not os.path.exists("results"):
    os.mkdir("results")

cwp_results.to_csv("results/cwp_by_region.csv", index=False)
felony_results.to_csv("results/felonies_by_region.csv", index=False)

## Felonies vs CWP

Create table comparing major figures from felonies and CWP tables.

In [13]:
#Selected columns to compare
filter_cols = ["Region", "Race/Ethnicity", "Total Count", 
                "Percent of Total", "% Bail Set or Remanded"]
filtered_cwp = cwp_results[filter_cols]
filtered_fel = felony_results[filter_cols]

fel_v_cwp = pd.merge(filtered_cwp, filtered_fel, 
                     on=["Region", "Race/Ethnicity"], 
                     suffixes=[" CWP", " Felonies"])

#Calculate percentage of felonies involving CWP charges
fel_v_cwp["% CWP of Felonies"] = fel_v_cwp["Total Count CWP"] / fel_v_cwp["Total Count Felonies"]
fel_v_cwp["% CWP of Felonies"] = fel_v_cwp["% CWP of Felonies"].apply(lambda n : f"{n:.2%}")

#Reorganize columns
ordered_cols = ["Region", "Race/Ethnicity", "% CWP of Felonies", "Total Count CWP", 
                "Total Count Felonies", "Percent of Total CWP", 
                "Percent of Total Felonies", "% Bail Set or Remanded CWP",
                "% Bail Set or Remanded Felonies"]

fel_v_cwp = fel_v_cwp.reindex(columns=ordered_cols)
fel_v_cwp.to_csv("results/CWP_vs_felonies.csv", index=False)